### First of all loading the libraries we will use

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor,GradientBoostingRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from sklearn.linear_model import Ridge, RidgeCV, Lasso
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
pd.set_option('display.max_columns', None)

### Loading our train dataset. With this dataset we will train our model.

In [10]:
data = pd.read_csv('train.csv')

In [11]:
df = data.copy()
df.head() # to see the first five rows of our data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### A quick look to our feature types and their null values

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [13]:
df.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

### These 3 features had many null values so I decided to drop them

In [14]:
df.drop(['PoolQC','Fence','MiscFeature'],axis=1,inplace=True)

In [15]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


### After a quick look to every feature one by one, this is how I decided to fill the null values on the remaining features

In [16]:
df['Alley'] = df['Alley'].fillna('No Alley')
df['MasVnrType'] = df['MasVnrType'].fillna('None')
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
df['BsmtQual'] = df['BsmtQual'].fillna('No Basement')
df['BsmtCond'] = df['BsmtCond'].fillna('No Basement Condition')
df['BsmtExposure'] = df['BsmtExposure'].fillna('No Basement Exposure')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('No Basement Type')
df['BsmtFinSF1'] = df['BsmtFinSF1'].fillna('No Basement')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('No Basement')
df['Electrical'] = df['Electrical'].fillna('None')
df['FireplaceQu'] = df['FireplaceQu'].fillna('None')
df['GarageType'] = df['GarageType'].fillna('None')
df['GarageFinish'] = df['GarageFinish'].fillna('None')
df['GarageQual'] = df['GarageQual'].fillna('None')
df['GarageCond'] = df['GarageCond'].fillna('None')
df['YrSold'] = df['YrSold'].astype(str) # We will use Year Sold as categorical value

### Splitting between X and y. 'ID' is not a feature that affects the price of a house that's why it was dropped

In [17]:
X = df.drop(['Id','SalePrice'],axis=1)
y = df['SalePrice']

### Splitting the data into train and test set. We will keep the 80% of our data for training and the rest 20% for testing

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42,shuffle=True)

### Finding our numerical data and use the rest as categorical

In [19]:
numerical_features = ['LotFrontage','MSSubClass','OverallCond','LotArea','OverallQual','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold']

In [20]:
categorical_features = [col for col in X_train.columns if col not in numerical_features]

### Creating Preprocessing Pipelines

At this stage, we build separate pipelines for **numerical** and **categorical** features.  

- For **numerical features**:  
  - We will impute missing values using **KNNImputer**.  
  - Then we standardize the data with **StandardScaler** to ensure all numerical variables are on a similar scale.  

- For **categorical features**:  
  - We will encode them using **OneHotEncoder** so that machine learning models can handle categorical variables properly.  

Finally, we will combine both pipelines into a single **ColumnTransformer**, which applies the appropriate transformations to each type of feature automatically.  

In [21]:
numeric_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler',StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [22]:
preprocessor = ColumnTransformer([
    ('numeric',numeric_pipeline,numerical_features),
    ('categorical',categorical_pipeline,categorical_features)
])

### Model Training & Comparison

We now test several regression models using the same preprocessing pipeline:  
- Linear Regression (baseline)  
- XGBoost  
- Random Forest  
- AdaBoost  
- Gradient Boosting 
- CatBoost

Each model is wrapped in a pipeline with the **preprocessor** (imputation, scaling, encoding) and then trained on the training set.  
We evaluate them on the test set using the **R² score** to compare performance.  

In [23]:
lr = LinearRegression()
xgb = XGBRegressor()
rf = RandomForestRegressor()
ada = AdaBoostRegressor()
gbr = GradientBoostingRegressor()
cat = CatBoostRegressor()

regs = [lr,xgb,rf,ada,gbr,cat]

for i in regs:
    pipe = Pipeline([
        ('preprocessor',preprocessor),
        ('regressor',i)
    ])
    pipe.fit(X_train,y_train)
    print(i,'score :',pipe.score(X_test, y_test))
    print('------------------------')

LinearRegression() score : 0.8843372729845601
------------------------
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...) score : 0.9142084717750549
------------------------
RandomForestRegressor() score : 0.8899572935057413
------------------------
AdaBoostRegressor() score : 0.831372826440744
-----

167:	learn: 17694.7614605	total: 756ms	remaining: 3.74s
168:	learn: 17652.9567496	total: 761ms	remaining: 3.74s
169:	learn: 17618.3989772	total: 764ms	remaining: 3.73s
170:	learn: 17569.2734903	total: 768ms	remaining: 3.72s
171:	learn: 17526.3918455	total: 772ms	remaining: 3.71s
172:	learn: 17506.7715244	total: 776ms	remaining: 3.71s
173:	learn: 17466.0621900	total: 780ms	remaining: 3.7s
174:	learn: 17422.7420166	total: 783ms	remaining: 3.69s
175:	learn: 17374.2081237	total: 787ms	remaining: 3.68s
176:	learn: 17329.0424973	total: 790ms	remaining: 3.67s
177:	learn: 17288.5303816	total: 794ms	remaining: 3.67s
178:	learn: 17255.0160664	total: 797ms	remaining: 3.65s
179:	learn: 17218.1989616	total: 800ms	remaining: 3.65s
180:	learn: 17183.5675388	total: 803ms	remaining: 3.63s
181:	learn: 17166.1817273	total: 807ms	remaining: 3.63s
182:	learn: 17151.3530185	total: 810ms	remaining: 3.62s
183:	learn: 17105.0856875	total: 813ms	remaining: 3.61s
184:	learn: 17051.6657940	total: 816ms	remaining:

348:	learn: 12171.5770704	total: 1.36s	remaining: 2.53s
349:	learn: 12141.3631335	total: 1.36s	remaining: 2.53s
350:	learn: 12118.5962247	total: 1.36s	remaining: 2.52s
351:	learn: 12092.9726785	total: 1.37s	remaining: 2.52s
352:	learn: 12070.2837494	total: 1.37s	remaining: 2.51s
353:	learn: 12046.1243541	total: 1.38s	remaining: 2.51s
354:	learn: 12028.0826123	total: 1.38s	remaining: 2.5s
355:	learn: 12008.9425406	total: 1.38s	remaining: 2.5s
356:	learn: 11983.9415027	total: 1.38s	remaining: 2.49s
357:	learn: 11956.7522672	total: 1.39s	remaining: 2.49s
358:	learn: 11932.8039843	total: 1.39s	remaining: 2.48s
359:	learn: 11916.5752159	total: 1.39s	remaining: 2.48s
360:	learn: 11903.8772068	total: 1.4s	remaining: 2.47s
361:	learn: 11877.8260446	total: 1.4s	remaining: 2.47s
362:	learn: 11845.5574356	total: 1.4s	remaining: 2.46s
363:	learn: 11823.2786555	total: 1.41s	remaining: 2.46s
364:	learn: 11799.1297864	total: 1.41s	remaining: 2.46s
365:	learn: 11784.7425541	total: 1.41s	remaining: 2.4

528:	learn: 9044.5605215	total: 1.95s	remaining: 1.74s
529:	learn: 9043.1775326	total: 1.95s	remaining: 1.73s
530:	learn: 9032.2549009	total: 1.96s	remaining: 1.73s
531:	learn: 9025.3112736	total: 1.96s	remaining: 1.73s
532:	learn: 9021.6039769	total: 1.96s	remaining: 1.72s
533:	learn: 9012.7877181	total: 1.97s	remaining: 1.72s
534:	learn: 9011.4419137	total: 1.97s	remaining: 1.71s
535:	learn: 9008.8725396	total: 1.97s	remaining: 1.71s
536:	learn: 8999.2896136	total: 1.98s	remaining: 1.71s
537:	learn: 8982.6546990	total: 1.98s	remaining: 1.7s
538:	learn: 8970.2811430	total: 1.98s	remaining: 1.7s
539:	learn: 8962.1628082	total: 1.99s	remaining: 1.69s
540:	learn: 8955.6068236	total: 1.99s	remaining: 1.69s
541:	learn: 8940.8322671	total: 1.99s	remaining: 1.69s
542:	learn: 8927.4268511	total: 2s	remaining: 1.68s
543:	learn: 8908.5906994	total: 2s	remaining: 1.68s
544:	learn: 8900.9799818	total: 2s	remaining: 1.67s
545:	learn: 8899.7143768	total: 2.01s	remaining: 1.67s
546:	learn: 8880.1107

709:	learn: 7327.8033300	total: 2.55s	remaining: 1.04s
710:	learn: 7315.7541728	total: 2.55s	remaining: 1.04s
711:	learn: 7307.7066651	total: 2.56s	remaining: 1.03s
712:	learn: 7300.5131947	total: 2.56s	remaining: 1.03s
713:	learn: 7292.7777736	total: 2.56s	remaining: 1.03s
714:	learn: 7285.6274276	total: 2.57s	remaining: 1.02s
715:	learn: 7274.1271569	total: 2.57s	remaining: 1.02s
716:	learn: 7264.6886195	total: 2.57s	remaining: 1.01s
717:	learn: 7258.3295464	total: 2.58s	remaining: 1.01s
718:	learn: 7247.7079072	total: 2.58s	remaining: 1.01s
719:	learn: 7233.7860992	total: 2.58s	remaining: 1s
720:	learn: 7226.5532992	total: 2.59s	remaining: 1s
721:	learn: 7215.1822671	total: 2.59s	remaining: 997ms
722:	learn: 7202.6814505	total: 2.59s	remaining: 994ms
723:	learn: 7188.5887984	total: 2.6s	remaining: 990ms
724:	learn: 7173.8789014	total: 2.6s	remaining: 986ms
725:	learn: 7166.9349407	total: 2.6s	remaining: 983ms
726:	learn: 7152.7756097	total: 2.61s	remaining: 979ms
727:	learn: 7140.80

888:	learn: 6066.5129414	total: 3.15s	remaining: 394ms
889:	learn: 6051.2045998	total: 3.16s	remaining: 390ms
890:	learn: 6041.0057998	total: 3.16s	remaining: 387ms
891:	learn: 6032.9178949	total: 3.16s	remaining: 383ms
892:	learn: 6032.1222206	total: 3.17s	remaining: 380ms
893:	learn: 6025.4640507	total: 3.17s	remaining: 376ms
894:	learn: 6019.8094648	total: 3.17s	remaining: 372ms
895:	learn: 6015.9911713	total: 3.18s	remaining: 369ms
896:	learn: 6008.8417580	total: 3.18s	remaining: 365ms
897:	learn: 6005.5608122	total: 3.18s	remaining: 362ms
898:	learn: 5993.9008615	total: 3.19s	remaining: 358ms
899:	learn: 5993.1629847	total: 3.19s	remaining: 355ms
900:	learn: 5992.5012901	total: 3.19s	remaining: 351ms
901:	learn: 5982.1867540	total: 3.2s	remaining: 347ms
902:	learn: 5975.3018645	total: 3.2s	remaining: 344ms
903:	learn: 5969.0150250	total: 3.2s	remaining: 340ms
904:	learn: 5963.5329382	total: 3.21s	remaining: 337ms
905:	learn: 5958.9676424	total: 3.21s	remaining: 333ms
906:	learn: 5

### Hyperparameter Tuning with GridSearchCV

To improve model performance, we perform hyperparameter tuning using **GridSearchCV**.  
We test two models — Gradient Boosting and XGBoost — across different values of `n_estimators` and `learning_rate`.  
GridSearchCV runs cross-validation for each parameter combination and selects the model with the **lowest error**.  

Finally, we print:
- The best regressor chosen,  
- Its optimal parameters,  
- The best cross-validation score.  

In [16]:
pipeline = Pipeline([
    ('preprocessor',preprocessor),
    ('clf', GradientBoostingRegressor()) 
])

parameters = [
    {   
        'clf': [GradientBoostingRegressor(random_state=42)],
        'clf__n_estimators': [100,150,200],
        'clf__learning_rate': [0.01,0.05]
    },
    
    { 
        'clf': [XGBRegressor(random_state=42)],
        'clf__n_estimators': [150,100,250],
        'clf__learning_rate': [0.05, 0.1]
    }
]

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=parameters,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)


print("Best Regressor:", type(grid_search.best_estimator_.named_steps['clf']).__name__)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Regressor: GradientBoostingRegressor
Best Parameters: {'clf': GradientBoostingRegressor(random_state=42), 'clf__learning_rate': 0.05, 'clf__n_estimators': 200}
Best Score: -852903862.5228751


The best cross-validation score is reported as a negative value because we used 
`scoring="neg_mean_squared_error"`.  
The actual MSE is **852,903,862**, which corresponds to an RMSE of about **29,200**.  
This means our model’s predictions are, on average, off by around 29K in house prices. Not ideal. 

Also we can put all of our results in a dataframe

In [17]:
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__learning_rate,param_clf__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,3.480815,0.168163,0.057026,0.009202,GradientBoostingRegressor(random_state=42),0.05,200,{'clf': GradientBoostingRegressor(random_state...,-1.014632e+09,-1.168632e+09,-1.024820e+09,-5.822022e+08,-4.742329e+08,-8.529039e+08,2.727881e+08,1
4,2.536152,0.059140,0.047900,0.003631,GradientBoostingRegressor(random_state=42),0.05,150,{'clf': GradientBoostingRegressor(random_state...,-1.040966e+09,-1.189385e+09,-1.075217e+09,-6.106641e+08,-4.848841e+08,-8.802234e+08,2.787104e+08,2
3,1.873553,0.121322,0.045060,0.004119,GradientBoostingRegressor(random_state=42),0.05,100,{'clf': GradientBoostingRegressor(random_state...,-1.123433e+09,-1.221989e+09,-1.184663e+09,-6.723639e+08,-5.072465e+08,-9.419391e+08,2.939086e+08,3
11,1.640120,0.040214,0.033925,0.005445,"XGBRegressor(base_score=None, booster=None, ca...",0.10,250,"{'clf': XGBRegressor(base_score=None, booster=...",-1.370109e+09,-1.221663e+09,-9.802380e+08,-6.154108e+08,-6.326261e+08,-9.640095e+08,3.042702e+08,4
9,1.174450,0.088064,0.065954,0.002381,"XGBRegressor(base_score=None, booster=None, ca...",0.10,150,"{'clf': XGBRegressor(base_score=None, booster=...",-1.371269e+09,-1.225995e+09,-9.870549e+08,-6.151890e+08,-6.328674e+08,-9.664750e+08,3.053942e+08,5
8,1.883184,0.158931,0.065588,0.005326,"XGBRegressor(base_score=None, booster=None, ca...",0.05,250,"{'clf': XGBRegressor(base_score=None, booster=...",-1.328266e+09,-1.341274e+09,-9.607986e+08,-5.947811e+08,-6.139351e+08,-9.678109e+08,3.267509e+08,6
10,0.937627,0.064604,0.048004,0.005788,"XGBRegressor(base_score=None, booster=None, ca...",0.10,100,"{'clf': XGBRegressor(base_score=None, booster=...",-1.363117e+09,-1.229827e+09,-9.895033e+08,-6.193715e+08,-6.386288e+08,-9.680895e+08,3.017152e+08,7
6,1.235320,0.088462,0.055017,0.004296,"XGBRegressor(base_score=None, booster=None, ca...",0.05,150,"{'clf': XGBRegressor(base_score=None, booster=...",-1.337412e+09,-1.352089e+09,-9.762655e+08,-6.071142e+08,-6.198413e+08,-9.785445e+08,3.270949e+08,8
7,0.926192,0.039979,0.054711,0.003396,"XGBRegressor(base_score=None, booster=None, ca...",0.05,100,"{'clf': XGBRegressor(base_score=None, booster=...",-1.398929e+09,-1.356088e+09,-1.024620e+09,-6.225514e+08,-6.406940e+08,-1.008576e+09,3.339909e+08,9
2,3.870818,0.081239,0.043397,0.002689,GradientBoostingRegressor(random_state=42),0.01,200,{'clf': GradientBoostingRegressor(random_state...,-1.399102e+09,-1.305068e+09,-1.822502e+09,-1.018869e+09,-8.172018e+08,-1.272549e+09,3.438652e+08,10


A very strong algorithm for categorical features is CatBoost and we will use it here due to the big amount of categorical features we have.
We define separate preprocessing pipelines again for numerical and categorical features 
to keep this section self-contained.  
These pipelines are combined into a `ColumnTransformer`, which is then used in the main pipeline 
alongside different regressors (Gradient Boosting and CatBoost) for hyperparameter tuning.  

In [57]:
num_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_features),
    ('cat', cat_pipeline, categorical_features)
])


pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor())
])

parameters = [
    {   
        'regressor': [GradientBoostingRegressor(random_state=42)],
        'regressor__n_estimators': [150,200,250],
        'regressor__learning_rate': [0.05,0.1]
    },
    
    { 
        'regressor': [CatBoostRegressor(verbose=0, random_state=42)],
        'regressor__iterations': [500, 1000],
        'regressor__learning_rate': [0.01,0.05,0.1],
        'regressor__depth': [6, 8, 10],
        'regressor__l2_leaf_reg': [3, 5, 7]
    }
]

grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=parameters,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)


print("Best Regressor:", type(grid_search.best_estimator_.named_steps['regressor']).__name__)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


C:\Users\anton\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
1 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\anton\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\anton\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\anton\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 473, in fit
    self._final_estimator.fit(

Best Regressor: CatBoostRegressor
Best Parameters: {'regressor': <catboost.core.CatBoostRegressor object at 0x00000190A90D7440>, 'regressor__depth': 6, 'regressor__iterations': 1000, 'regressor__l2_leaf_reg': 3, 'regressor__learning_rate': 0.05}
Best Score: -682500326.1464713


### Our final result.

Finally we have our much improved result with CatBoostRegressor.

The actual MSE is **682,500,326**, which corresponds to an RMSE of about **26,100**.  
This shows our tuned model improves prediction accuracy, with an average error of ~26K in house prices. 
It is not ideal yet, but this will be my first submission in the competition.

In [58]:
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor,param_regressor__learning_rate,param_regressor__n_estimators,param_regressor__depth,param_regressor__iterations,param_regressor__l2_leaf_reg,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
16,41.379753,0.534399,0.226606,0.043773,<catboost.core.CatBoostRegressor object at 0x0...,0.05,NaN,6.0,1000.0,3.0,{'regressor': <catboost.core.CatBoostRegressor...,-6.945461e+08,-8.285491e+08,-9.205807e+08,-5.470035e+08,-4.218223e+08,-6.825003e+08,1.813670e+08,1
7,16.749523,2.802958,0.193309,0.016240,<catboost.core.CatBoostRegressor object at 0x0...,0.05,NaN,6.0,500.0,3.0,{'regressor': <catboost.core.CatBoostRegressor...,-7.064695e+08,-8.430980e+08,-9.352670e+08,-5.496121e+08,-4.331460e+08,-6.935185e+08,1.841124e+08,2
23,46.402933,1.544825,0.315923,0.037249,<catboost.core.CatBoostRegressor object at 0x0...,0.10,NaN,6.0,1000.0,7.0,{'regressor': <catboost.core.CatBoostRegressor...,-8.180941e+08,-8.746361e+08,-8.889789e+08,-5.620132e+08,-4.382267e+08,-7.163898e+08,1.824168e+08,3
14,20.665633,0.305412,0.196335,0.007081,<catboost.core.CatBoostRegressor object at 0x0...,0.10,NaN,6.0,500.0,7.0,{'regressor': <catboost.core.CatBoostRegressor...,-8.161818e+08,-8.797839e+08,-8.973970e+08,-5.571174e+08,-4.459735e+08,-7.192907e+08,1.832322e+08,4
40,261.508940,119.492221,0.630808,0.140155,<catboost.core.CatBoostRegressor object at 0x0...,0.05,NaN,8.0,1000.0,7.0,{'regressor': <catboost.core.CatBoostRegressor...,-8.364205e+08,-8.284770e+08,-9.252044e+08,-5.501891e+08,-4.687118e+08,-7.218006e+08,1.785470e+08,5
19,40.499566,0.371360,0.210462,0.017293,<catboost.core.CatBoostRegressor object at 0x0...,0.05,NaN,6.0,1000.0,5.0,{'regressor': <catboost.core.CatBoostRegressor...,-7.827705e+08,-9.053482e+08,-9.471633e+08,-5.453116e+08,-4.437149e+08,-7.248617e+08,1.983071e+08,6
17,40.939644,0.455764,0.210560,0.009113,<catboost.core.CatBoostRegressor object at 0x0...,0.10,NaN,6.0,1000.0,3.0,{'regressor': <catboost.core.CatBoostRegressor...,-7.910690e+08,-9.149176e+08,-8.832874e+08,-6.120226e+08,-4.335453e+08,-7.269684e+08,1.806539e+08,7
34,99.334226,1.707107,0.391778,0.049461,<catboost.core.CatBoostRegressor object at 0x0...,0.05,NaN,8.0,1000.0,3.0,{'regressor': <catboost.core.CatBoostRegressor...,-8.043068e+08,-7.552222e+08,-1.009013e+09,-5.980281e+08,-4.820508e+08,-7.297242e+08,1.804998e+08,8
8,20.037982,0.664683,0.205840,0.015916,<catboost.core.CatBoostRegressor object at 0x0...,0.10,NaN,6.0,500.0,3.0,{'regressor': <catboost.core.CatBoostRegressor...,-7.917336e+08,-9.224828e+08,-8.839767e+08,-6.153348e+08,-4.353897e+08,-7.297835e+08,1.813905e+08,9
37,100.221231,1.165589,0.384881,0.036768,<catboost.core.CatBoostRegressor object at 0x0...,0.05,NaN,8.0,1000.0,5.0,{'regressor': <catboost.core.CatBoostRegressor...,-8.207748e+08,-8.084231e+08,-9.731514e+08,-6.049162e+08,-4.700576e+08,-7.354646e+08,1.769630e+08,10


In [59]:
model = CatBoostRegressor(random_state=42,iterations=1000,l2_leaf_reg=3,learning_rate=0.05,depth=6)

In [61]:
num_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_features),
    ('cat', cat_pipeline, categorical_features)
])


pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', model)
])

In [62]:
pipe.fit(X_train,y_train)

0:	learn: 74862.7498469	total: 4.87ms	remaining: 4.86s
1:	learn: 72517.4681592	total: 9.25ms	remaining: 4.62s
2:	learn: 70587.6867870	total: 13.3ms	remaining: 4.42s
3:	learn: 68419.3290683	total: 17.3ms	remaining: 4.3s
4:	learn: 66288.6512383	total: 21.5ms	remaining: 4.27s
5:	learn: 64319.6033396	total: 25.6ms	remaining: 4.24s
6:	learn: 62499.5939995	total: 29.7ms	remaining: 4.21s
7:	learn: 60866.5656917	total: 33.8ms	remaining: 4.19s
8:	learn: 59263.2345107	total: 37.8ms	remaining: 4.16s
9:	learn: 57573.1117654	total: 41.5ms	remaining: 4.11s
10:	learn: 55964.3492991	total: 45.2ms	remaining: 4.06s
11:	learn: 54587.4858528	total: 48.9ms	remaining: 4.03s
12:	learn: 53216.3318535	total: 52.7ms	remaining: 4s
13:	learn: 52041.2775017	total: 56.5ms	remaining: 3.98s
14:	learn: 50842.1778851	total: 60.1ms	remaining: 3.95s
15:	learn: 49508.9827959	total: 63.8ms	remaining: 3.92s
16:	learn: 48378.6711664	total: 67.8ms	remaining: 3.92s
17:	learn: 47306.2084650	total: 71.4ms	remaining: 3.9s
18:	lea

161:	learn: 16662.1146052	total: 619ms	remaining: 3.2s
162:	learn: 16627.2849505	total: 623ms	remaining: 3.2s
163:	learn: 16577.4491118	total: 627ms	remaining: 3.2s
164:	learn: 16555.8114385	total: 630ms	remaining: 3.19s
165:	learn: 16495.2138762	total: 634ms	remaining: 3.18s
166:	learn: 16452.6072058	total: 637ms	remaining: 3.18s
167:	learn: 16408.2567167	total: 641ms	remaining: 3.17s
168:	learn: 16358.0217240	total: 645ms	remaining: 3.17s
169:	learn: 16310.9590207	total: 649ms	remaining: 3.17s
170:	learn: 16288.2835386	total: 653ms	remaining: 3.16s
171:	learn: 16241.6450013	total: 657ms	remaining: 3.16s
172:	learn: 16192.5099727	total: 660ms	remaining: 3.16s
173:	learn: 16140.0360233	total: 664ms	remaining: 3.15s
174:	learn: 16097.1528365	total: 668ms	remaining: 3.15s
175:	learn: 16055.1769869	total: 672ms	remaining: 3.14s
176:	learn: 16027.0006812	total: 675ms	remaining: 3.14s
177:	learn: 16008.5673974	total: 677ms	remaining: 3.13s
178:	learn: 15995.6265530	total: 680ms	remaining: 3

320:	learn: 11907.4156688	total: 1.23s	remaining: 2.6s
321:	learn: 11904.2236814	total: 1.24s	remaining: 2.6s
322:	learn: 11870.8805485	total: 1.24s	remaining: 2.6s
323:	learn: 11837.2154657	total: 1.24s	remaining: 2.59s
324:	learn: 11814.8085711	total: 1.25s	remaining: 2.59s
325:	learn: 11806.4660299	total: 1.25s	remaining: 2.58s
326:	learn: 11778.1112799	total: 1.25s	remaining: 2.58s
327:	learn: 11758.8755786	total: 1.26s	remaining: 2.57s
328:	learn: 11738.6626116	total: 1.26s	remaining: 2.57s
329:	learn: 11719.0135530	total: 1.26s	remaining: 2.57s
330:	learn: 11690.7903124	total: 1.27s	remaining: 2.56s
331:	learn: 11666.9816245	total: 1.27s	remaining: 2.56s
332:	learn: 11652.3138649	total: 1.27s	remaining: 2.55s
333:	learn: 11624.8973026	total: 1.28s	remaining: 2.55s
334:	learn: 11602.9062289	total: 1.28s	remaining: 2.55s
335:	learn: 11581.1836293	total: 1.29s	remaining: 2.54s
336:	learn: 11556.0602760	total: 1.29s	remaining: 2.54s
337:	learn: 11549.7209950	total: 1.29s	remaining: 2

480:	learn: 9031.5671970	total: 1.83s	remaining: 1.98s
481:	learn: 9017.5320850	total: 1.83s	remaining: 1.97s
482:	learn: 9016.1537962	total: 1.84s	remaining: 1.97s
483:	learn: 9003.9436293	total: 1.84s	remaining: 1.96s
484:	learn: 8992.6266368	total: 1.85s	remaining: 1.96s
485:	learn: 8991.2855311	total: 1.85s	remaining: 1.96s
486:	learn: 8985.7510765	total: 1.85s	remaining: 1.95s
487:	learn: 8966.4712188	total: 1.86s	remaining: 1.95s
488:	learn: 8951.0651540	total: 1.86s	remaining: 1.95s
489:	learn: 8935.8889194	total: 1.86s	remaining: 1.94s
490:	learn: 8921.4323179	total: 1.87s	remaining: 1.94s
491:	learn: 8902.4211633	total: 1.87s	remaining: 1.93s
492:	learn: 8882.4455604	total: 1.88s	remaining: 1.93s
493:	learn: 8864.1200777	total: 1.88s	remaining: 1.93s
494:	learn: 8841.0234195	total: 1.89s	remaining: 1.92s
495:	learn: 8839.5527818	total: 1.89s	remaining: 1.92s
496:	learn: 8837.5732948	total: 1.89s	remaining: 1.92s
497:	learn: 8819.9872561	total: 1.9s	remaining: 1.91s
498:	learn:

642:	learn: 7193.0636569	total: 2.45s	remaining: 1.36s
643:	learn: 7178.0453469	total: 2.45s	remaining: 1.36s
644:	learn: 7168.9882491	total: 2.46s	remaining: 1.35s
645:	learn: 7151.0649757	total: 2.46s	remaining: 1.35s
646:	learn: 7137.9827576	total: 2.46s	remaining: 1.34s
647:	learn: 7132.5753605	total: 2.47s	remaining: 1.34s
648:	learn: 7120.6713916	total: 2.47s	remaining: 1.34s
649:	learn: 7116.3154731	total: 2.48s	remaining: 1.33s
650:	learn: 7109.0394766	total: 2.48s	remaining: 1.33s
651:	learn: 7094.3741467	total: 2.48s	remaining: 1.32s
652:	learn: 7087.4137012	total: 2.49s	remaining: 1.32s
653:	learn: 7086.3135984	total: 2.49s	remaining: 1.32s
654:	learn: 7076.6736654	total: 2.49s	remaining: 1.31s
655:	learn: 7070.1211437	total: 2.5s	remaining: 1.31s
656:	learn: 7061.6106130	total: 2.5s	remaining: 1.31s
657:	learn: 7057.5787028	total: 2.51s	remaining: 1.3s
658:	learn: 7043.1249349	total: 2.51s	remaining: 1.3s
659:	learn: 7027.1417715	total: 2.51s	remaining: 1.29s
660:	learn: 70

792:	learn: 5913.8279033	total: 3.05s	remaining: 796ms
793:	learn: 5905.2227297	total: 3.05s	remaining: 793ms
794:	learn: 5901.0460308	total: 3.06s	remaining: 789ms
795:	learn: 5889.9515485	total: 3.06s	remaining: 785ms
796:	learn: 5885.3082690	total: 3.07s	remaining: 781ms
797:	learn: 5880.0366634	total: 3.07s	remaining: 777ms
798:	learn: 5866.7841327	total: 3.07s	remaining: 773ms
799:	learn: 5858.5374409	total: 3.08s	remaining: 770ms
800:	learn: 5851.3392554	total: 3.08s	remaining: 766ms
801:	learn: 5849.3793154	total: 3.09s	remaining: 762ms
802:	learn: 5836.1853147	total: 3.09s	remaining: 758ms
803:	learn: 5827.2265826	total: 3.09s	remaining: 754ms
804:	learn: 5820.4426450	total: 3.1s	remaining: 750ms
805:	learn: 5812.8522987	total: 3.1s	remaining: 746ms
806:	learn: 5807.6324353	total: 3.1s	remaining: 742ms
807:	learn: 5795.6551258	total: 3.11s	remaining: 739ms
808:	learn: 5784.7991030	total: 3.11s	remaining: 735ms
809:	learn: 5777.8822370	total: 3.12s	remaining: 731ms
810:	learn: 5

995:	learn: 4665.8807711	total: 3.85s	remaining: 15.5ms
996:	learn: 4660.0337648	total: 3.86s	remaining: 11.6ms
997:	learn: 4655.1962656	total: 3.86s	remaining: 7.74ms
998:	learn: 4649.3611713	total: 3.87s	remaining: 3.87ms
999:	learn: 4643.1856744	total: 3.87s	remaining: 0us


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['LotFrontage', 'MSSubClass',
                                                   'OverallCond', 'LotArea',
                                                   'OverallQual', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   '...
                                                   'Neighborhood', 'Condition1',
                                                   'Condition2', 'BldgType',
                                                   'HouseStyle', 'RoofStyle',
                                                   'RoofMatl', 'Exterior1st',
                                                   'Exterior2nd', 'MasVnrType',
                                                   'ExterQual', 'ExterCond',
                                                   'Foundation', 'BsmtQual',
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', ...])])),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x00000190A98852E0>)])

### Saving and Reloading the Trained Pipeline

After finding the best parameters and fitting the final pipeline,  
we save the model using `pickle`.  
This allows us to reload the trained pipeline later and make predictions directly,  
without having to retrain the model each time.  

In [64]:
pickle.dump(pipe, open('cat_pipeline.pickle', 'wb'))